In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
device = torch.device("cuda")
    
    
batch_size = 128
model_name = "Resnet_20_quant_project"
model = resnet20_quant_project()


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
print_freq = 100
def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch,adjust_list):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1

In [3]:
lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=weight_decay)

if not os.path.exists('result'): 
    os.makedirs('result')

fdir = 'result/'+str(model_name) 
if not os.path.exists(fdir):
    os.makedirs(fdir)

adjust_list = [80,90]

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch,adjust_list)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.697 (0.697)	Data 0.576 (0.576)	Loss 2.6520 (2.6520)	Prec 6.250% (6.250%)
Epoch: [0][100/391]	Time 0.039 (0.052)	Data 0.003 (0.008)	Loss 1.9711 (2.0783)	Prec 25.000% (21.744%)
Epoch: [0][200/391]	Time 0.041 (0.049)	Data 0.002 (0.005)	Loss 1.8291 (1.9321)	Prec 33.594% (27.433%)
Epoch: [0][300/391]	Time 0.046 (0.049)	Data 0.002 (0.004)	Loss 1.6122 (1.8335)	Prec 39.062% (31.364%)
Validation starts
Test: [0/79]	Time 0.585 (0.585)	Loss 1.6914 (1.6914)	Prec 35.938% (35.938%)
 * Prec 37.970% 
best acc: 37.970000
Epoch: [1][0/391]	Time 0.536 (0.536)	Data 0.479 (0.479)	Loss 1.5299 (1.5299)	Prec 44.531% (44.531%)
Epoch: [1][100/391]	Time 0.051 (0.052)	Data 0.002 (0.007)	Loss 1.4429 (1.4517)	Prec 47.656% (47.300%)
Epoch: [1][200/391]	Time 0.048 (0.049)	Data 0.002 (0.005)	Loss 1.3430 (1.4097)	Prec 48.438% (49.114%)
Epoch: [1][300/391]	Time 0.047 (0.048)	Data 0.002 (0.004)	Loss 1.1828 (1.3801)	Prec 54.688% (50.016%)
Validation starts
Test: [0/79]	Time 0.441 (0.441)	Loss 1.72

Epoch: [15][200/391]	Time 0.052 (0.048)	Data 0.002 (0.005)	Loss 0.5816 (0.5466)	Prec 82.031% (80.889%)
Epoch: [15][300/391]	Time 0.050 (0.048)	Data 0.003 (0.004)	Loss 0.4391 (0.5470)	Prec 83.594% (80.824%)
Validation starts
Test: [0/79]	Time 0.796 (0.796)	Loss 0.5999 (0.5999)	Prec 77.344% (77.344%)
 * Prec 78.220% 
best acc: 78.220000
Epoch: [16][0/391]	Time 0.641 (0.641)	Data 0.595 (0.595)	Loss 0.4703 (0.4703)	Prec 84.375% (84.375%)
Epoch: [16][100/391]	Time 0.056 (0.053)	Data 0.003 (0.008)	Loss 0.5447 (0.5199)	Prec 78.906% (81.877%)
Epoch: [16][200/391]	Time 0.045 (0.051)	Data 0.002 (0.005)	Loss 0.6851 (0.5185)	Prec 75.000% (82.027%)
Epoch: [16][300/391]	Time 0.044 (0.049)	Data 0.002 (0.004)	Loss 0.5540 (0.5217)	Prec 80.469% (81.842%)
Validation starts
Test: [0/79]	Time 0.245 (0.245)	Loss 0.6395 (0.6395)	Prec 74.219% (74.219%)
 * Prec 75.680% 
best acc: 78.220000
Epoch: [17][0/391]	Time 0.591 (0.591)	Data 0.527 (0.527)	Loss 0.5065 (0.5065)	Prec 83.594% (83.594%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.046 (0.048)	Data 0.002 (0.005)	Loss 0.4876 (0.4106)	Prec 80.469% (85.670%)
Validation starts
Test: [0/79]	Time 0.339 (0.339)	Loss 0.4842 (0.4842)	Prec 82.031% (82.031%)
 * Prec 82.590% 
best acc: 82.590000
Epoch: [31][0/391]	Time 1.075 (1.075)	Data 1.026 (1.026)	Loss 0.3488 (0.3488)	Prec 90.625% (90.625%)
Epoch: [31][100/391]	Time 0.045 (0.059)	Data 0.002 (0.012)	Loss 0.3139 (0.3979)	Prec 88.281% (86.123%)
Epoch: [31][200/391]	Time 0.034 (0.053)	Data 0.002 (0.007)	Loss 0.4960 (0.3981)	Prec 82.031% (86.023%)
Epoch: [31][300/391]	Time 0.037 (0.048)	Data 0.001 (0.005)	Loss 0.3585 (0.4020)	Prec 85.156% (85.922%)
Validation starts
Test: [0/79]	Time 0.384 (0.384)	Loss 0.4215 (0.4215)	Prec 87.500% (87.500%)
 * Prec 79.750% 
best acc: 82.590000
Epoch: [32][0/391]	Time 0.957 (0.957)	Data 0.914 (0.914)	Loss 0.3458 (0.3458)	Prec 85.938% (85.938%)
Epoch: [32][100/391]	Time 0.043 (0.056)	Data 0.002 (0.011)	Loss 0.5772 (0.4068)	Prec 77.344% (85.953%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.528 (0.528)	Loss 0.4761 (0.4761)	Prec 85.156% (85.156%)
 * Prec 83.370% 
best acc: 84.470000
Epoch: [46][0/391]	Time 0.860 (0.860)	Data 0.799 (0.799)	Loss 0.3677 (0.3677)	Prec 87.500% (87.500%)
Epoch: [46][100/391]	Time 0.049 (0.056)	Data 0.002 (0.010)	Loss 0.4591 (0.3518)	Prec 83.594% (87.438%)
Epoch: [46][200/391]	Time 0.046 (0.051)	Data 0.002 (0.006)	Loss 0.4097 (0.3537)	Prec 86.719% (87.352%)
Epoch: [46][300/391]	Time 0.052 (0.049)	Data 0.002 (0.005)	Loss 0.3269 (0.3565)	Prec 89.844% (87.344%)
Validation starts
Test: [0/79]	Time 0.522 (0.522)	Loss 0.4968 (0.4968)	Prec 82.031% (82.031%)
 * Prec 80.740% 
best acc: 84.470000
Epoch: [47][0/391]	Time 0.603 (0.603)	Data 0.521 (0.521)	Loss 0.3529 (0.3529)	Prec 87.500% (87.500%)
Epoch: [47][100/391]	Time 0.052 (0.053)	Data 0.003 (0.007)	Loss 0.3608 (0.3433)	Prec 89.062% (87.871%)
Epoch: [47][200/391]	Time 0.046 (0.052)	Data 0.003 (0.005)	Loss 0.3699 (0.3537)	Prec 82.812% (87.395%)
Epoch: [47][300/391]	

 * Prec 85.510% 
best acc: 85.510000
Epoch: [61][0/391]	Time 0.534 (0.534)	Data 0.477 (0.477)	Loss 0.3589 (0.3589)	Prec 87.500% (87.500%)
Epoch: [61][100/391]	Time 0.052 (0.052)	Data 0.003 (0.007)	Loss 0.3037 (0.3034)	Prec 88.281% (89.217%)
Epoch: [61][200/391]	Time 0.044 (0.048)	Data 0.002 (0.005)	Loss 0.2808 (0.3073)	Prec 89.062% (89.125%)
Epoch: [61][300/391]	Time 0.043 (0.047)	Data 0.002 (0.004)	Loss 0.3935 (0.3110)	Prec 88.281% (89.024%)
Validation starts
Test: [0/79]	Time 0.488 (0.488)	Loss 0.3072 (0.3072)	Prec 89.062% (89.062%)
 * Prec 85.460% 
best acc: 85.510000
Epoch: [62][0/391]	Time 0.604 (0.604)	Data 0.509 (0.509)	Loss 0.2207 (0.2207)	Prec 93.750% (93.750%)
Epoch: [62][100/391]	Time 0.036 (0.050)	Data 0.002 (0.007)	Loss 0.3309 (0.2777)	Prec 90.625% (90.114%)
Epoch: [62][200/391]	Time 0.050 (0.046)	Data 0.002 (0.005)	Loss 0.3072 (0.2872)	Prec 90.625% (89.871%)
Epoch: [62][300/391]	Time 0.038 (0.047)	Data 0.002 (0.004)	Loss 0.2786 (0.2982)	Prec 92.188% (89.525%)
Validation s

Epoch: [76][100/391]	Time 0.050 (0.052)	Data 0.003 (0.007)	Loss 0.2463 (0.2687)	Prec 89.844% (90.733%)
Epoch: [76][200/391]	Time 0.058 (0.049)	Data 0.003 (0.005)	Loss 0.3314 (0.2784)	Prec 89.062% (90.314%)
Epoch: [76][300/391]	Time 0.043 (0.047)	Data 0.002 (0.004)	Loss 0.2450 (0.2789)	Prec 89.062% (90.207%)
Validation starts
Test: [0/79]	Time 0.509 (0.509)	Loss 0.5306 (0.5306)	Prec 85.938% (85.938%)
 * Prec 83.070% 
best acc: 85.510000
Epoch: [77][0/391]	Time 0.747 (0.747)	Data 0.682 (0.682)	Loss 0.1254 (0.1254)	Prec 98.438% (98.438%)
Epoch: [77][100/391]	Time 0.041 (0.050)	Data 0.002 (0.009)	Loss 0.2512 (0.2830)	Prec 95.312% (89.859%)
Epoch: [77][200/391]	Time 0.042 (0.047)	Data 0.002 (0.005)	Loss 0.4202 (0.2913)	Prec 85.156% (89.564%)
Epoch: [77][300/391]	Time 0.046 (0.046)	Data 0.002 (0.004)	Loss 0.2912 (0.2918)	Prec 90.625% (89.665%)
Validation starts
Test: [0/79]	Time 0.395 (0.395)	Loss 0.3673 (0.3673)	Prec 86.719% (86.719%)
 * Prec 84.760% 
best acc: 85.510000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.046 (0.049)	Data 0.002 (0.005)	Loss 0.0951 (0.1391)	Prec 95.312% (95.293%)
Epoch: [91][300/391]	Time 0.039 (0.045)	Data 0.002 (0.004)	Loss 0.1328 (0.1375)	Prec 96.875% (95.297%)
Validation starts
Test: [0/79]	Time 0.545 (0.545)	Loss 0.2551 (0.2551)	Prec 92.969% (92.969%)
 * Prec 88.870% 
best acc: 88.900000
Epoch: [92][0/391]	Time 0.523 (0.523)	Data 0.474 (0.474)	Loss 0.0858 (0.0858)	Prec 97.656% (97.656%)
Epoch: [92][100/391]	Time 0.061 (0.052)	Data 0.002 (0.007)	Loss 0.1292 (0.1348)	Prec 95.312% (95.514%)
Epoch: [92][200/391]	Time 0.041 (0.050)	Data 0.002 (0.005)	Loss 0.1656 (0.1361)	Prec 93.750% (95.460%)
Epoch: [92][300/391]	Time 0.044 (0.049)	Data 0.002 (0.004)	Loss 0.1911 (0.1356)	Prec 95.312% (95.476%)
Validation starts
Test: [0/79]	Time 0.372 (0.372)	Loss 0.2372 (0.2372)	Prec 93.750% (93.750%)
 * Prec 89.000% 
best acc: 89.000000
Epoch: [93][0/391]	Time 1.364 (1.364)	Data 1.301 (1.301)	Loss 0.1841 (0.1841)	Prec 93.750% (93.750%)
Epoch: [93][100/391]	

In [3]:
PATH = "result/Resnet_20_quant_project/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8922/10000 (89%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)

7 -th layer prehooked
9 -th layer prehooked
13 -th layer prehooked
15 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
27 -th layer prehooked
34 -th layer prehooked
36 -th layer prehooked
42 -th layer prehooked
46 -th layer prehooked
48 -th layer prehooked
54 -th layer prehooked
56 -th layer prehooked
63 -th layer prehooked
65 -th layer prehooked
71 -th layer prehooked
75 -th layer prehooked
77 -th layer prehooked
83 -th layer prehooked
85 -th layer prehooked


In [5]:
## Layer 9
## Layer 13
## save_outputs.output[1][0]
## save_outputs.output[2][0]
model.layer1[0].conv2.weight_quant.wgt_alpha

Parameter containing:
tensor(2.6745, device='cuda:0', requires_grad=True)

In [6]:
w_bit = 4
weight_q = model.layer1[0].conv2.weight_q
w_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta

x_bit = 4
x = save_output.outputs[1][0]
x_alpha = model.layer1[0].conv2.act_alpha
x_delta = x_alpha/(2**x_bit-1)
act_quant = act_quantization(x_bit)
x_q = act_quant(x,x_alpha)
x_int = x_q/x_delta

conv_int = nn.Conv2d(8,8,kernel_size=3, padding=1, bias=False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
output_int = conv_int(x_int)
psum_recovered = output_int*w_delta*x_delta
relu = nn.ReLU(inplace=True)
psum_after_relu = relu(psum_recovered)

difference = (save_output.outputs[2][0] - psum_after_relu).mean()

print("The difference between psum original and psum recovered = {}".format(difference))

The difference between psum original and psum recovered = -5.591659828496631e-07


In [8]:
act_int = x_int[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [64, 32, 32]

# conv_int.weight.size() = torch.Size([64, 64, 3, 3])  <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij

padding = 1
stride = 1
array_size = 8
x_size = x_int.size()
nig = range(x_size[2])
njg = range(x_size[3])

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

a_pad = torch.zeros((x_size[1],x_size[2]+2*padding,x_size[3]+2*padding))

a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = act_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))

ic_tile = range(int(int(act_int.size(0))/array_size))
oc_tile = range(int(int(w_int.size(0))/array_size))

a_tile = torch.zeros(len(ic_tile),array_size,len(nig)+padding*2,len(njg)+padding*2).cuda()
a_tile = torch.reshape(a_tile,(a_tile.size(0),a_tile.size(1),-1))

for ict in ic_tile:
    a_tile[ict,:,:] = a_pad[(ict*array_size):((ict+1)*array_size),:]

w_tile = torch.zeros(len(ic_tile),len(oc_tile),array_size,array_size,len(kijg)).cuda()

for oct in oc_tile:
    for ict in ic_tile:
        w_tile[ict,oct,:,:,:] = w_int[(oct*array_size):((oct+1)*array_size),(ict*array_size):((ict+1)*array_size),:]

p_nijg = range(a_pad.size(1)) ## paded activation's nij group

psum = torch.zeros(len(ic_tile),len(oc_tile),array_size,len(p_nijg),len(kijg)).cuda() 

for kij in kijg:
    for ict in ic_tile:
        for oct in oc_tile:
            for nij in p_nijg:     # time domain, sequentially given input
                m = nn.Linear(array_size, array_size, bias=False)
                m.weight = torch.nn.Parameter(w_tile[ict,oct,:,:,kij])
                psum[ict,oct,:, nij, kij] = m(a_tile[ict,:,nij]).cuda()

In [9]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32 + 2*pad = 34

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1) #34 - 2 - 1 + 1 = 32
o_nijg = range(o_ni_dim**2)    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:  
        for ict in ic_tile:
            for oct in oc_tile:
                out[oct*array_size:(oct+1)*array_size,o_nij] = out[oct*array_size:(oct+1)*array_size,o_nij] + \
                psum[ict,oct,:, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 2nd index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

In [143]:
## dump all the files

## Helper functions to dump files
def dec_to_bin(arr,bit):
    bin_arr = []
    for a in arr:
        a = int(a)
        if a < 0:
            a+=bit
        b = '{0:04b}'.format(int(a))
        bin_arr.append(b)
    return bin_arr

def dec_to_bin_psum(arr,bit):
    bin_arr = []
    for a in arr:
        a = int(a)
        sign = 0
        if a < 0:
            sign = 1
            a+=bit
        b = '{0:16b}'.format(int(a))
        if sign == 1:
            b = b.replace(" ", "1")
        else:
            b = b.replace(" ","0")
        bin_arr.append(b)
    return bin_arr


def convert_to_list(arr):
    return arr.tolist()

In [144]:
## Activation dump

fp_act = open('Renet_activation_project.txt','w')
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act_dec = open('Resnet_activation_dec.txt','w')
for i in range(a_pad.size()[1]):
    act_line = a_pad[:,i]
    act_arr = [int(j+0.001) for j in convert_to_list(act_line)]
    for a in act_arr[::-1]:
        #print(int(a))
        fp_act_dec.write(str(int(a)))
        fp_act_dec.write(" ")
    fp_act_dec.write("\n")
    bin_act = dec_to_bin(act_arr,16)
    for b in bin_act[::-1]:
        #print(b)
        fp_act.write(b)
    fp_act.write('\n')
    

fp_act.close()
fp_act_dec.close()

In [145]:
## Weight dump
fp_wgt = open('Resnet_weight_project.txt','w')
fp_wgt.write("#####\n")
fp_wgt.write("#####\n")
fp_wgt.write("#####\n")
fp_wgt_dec = open('Resnet_weight_dec.txt','w')

for kij in range(9):
    for w in range(8):
        w_line = w_int[w,:,kij]
        w_arr = []
        for i in convert_to_list(w_line):
            if i < 0:
                w_arr.append(int(i-0.001))
            else:
                w_arr.append(int(i+0.001))
        
        for ww in w_arr[::-1]:
            #print(int(ww))
            fp_wgt_dec.write(str(int(ww)))
            fp_wgt_dec.write(" ")
        fp_wgt_dec.write("\n")
        bin_wgt = dec_to_bin(w_arr,16)
        for b in bin_wgt[::-1]:
            #print("Count = {} , b = {}".format(count,b))
            fp_wgt.write(b)
        fp_wgt.write('\n')

fp_wgt.close()
fp_wgt_dec.close()

In [149]:
## psum dump and output.txt dump
fp_psum = open('Resnet_psum_project.txt','w')
fp_psum.write("#####\n")
fp_psum.write("#####\n")
fp_psum.write("#####\n")

fp_psum_dec = open('Resnet_psum_dec.txt','w')
fp_psum_relu = open('Resnet_output_project.txt','w')
fp_psum_relu.write("#\n")
fp_psum_relu.write("#\n")
fp_psum_relu.write("#\n")
for kij in range(o_ni_dim*o_ni_dim):
    psum_line = out[:,kij]
    psum_arr = []
    out_arr = []
    for i in convert_to_list(psum_line):
        if i < 0:
            psum_arr.append(int(i-0.001))
            out_arr.append(0)
        else:
            psum_arr.append(int(i+0.001))
            out_arr.append(int(i+0.001))
    for p in psum_arr:
        #print(int(a))
        fp_psum_dec.write(str(int(p)))
        fp_psum_dec.write(" ")
    fp_psum_dec.write("\n")
    bin_act = dec_to_bin_psum(psum_arr,65536)
    out_bin_act = dec_to_bin_psum(out_arr,65536)
    for b in bin_act:
        #print(b)
        fp_psum.write(b)
    fp_psum.write('\n')
    
    for b in out_bin_act:
        fp_psum_relu.write(b)
    fp_psum_relu.write("\n")
fp_psum.close()
fp_psum_dec.close()
fp_psum_relu.close()

In [10]:
## For input activations tiling. Handled in testbench. Written here just for reference logic
our_hardware = 6
hor_step = our_hardware-ki_dim+1 ## 4
hor_step_mod = our_hardware
ver_step = (our_hardware-ki_dim+1)*a_pad_ni_dim ## 136
stop_point = (a_pad_ni_dim-our_hardware)*a_pad_ni_dim+1 ## 
group_count = 0
act_group = []
for v in range(0,stop_point,ver_step):
    ## Now move horizontally
    for h in range(v,v+34,hor_step):
        if h+our_hardware>v+a_pad_ni_dim:
            break
        group = []
        group_count+=1
        for hh in range(h,h+(our_hardware)*a_pad_ni_dim,a_pad_ni_dim):
            
            for hhh in range(our_hardware):
                group.append(hh+hhh)
        act_group.append(group)       
print(group_count)        
        

64


In [15]:
## For output tiling. Handled in testbench. Written here just for reference logic
our_out_hw = 4
out_ni_dim = int(math.sqrt(out.size(1)))
out_stop_point = (out_ni_dim-our_out_hw)*out_ni_dim+1
out_ver_step = our_out_hw*out_ni_dim
group_count = 0
out_group = []
for v in range(0,out_stop_point,out_ver_step):
    ## Now move horizontally
    for h in range(v,v+32,our_out_hw):
        group = []
        group_count+=1
        for hh in range(h,h+our_out_hw*out_ni_dim,out_ni_dim):
            for hhh in range(our_out_hw):
                group.append(hh+hhh)
        out_group.append(group)
